# Lesson 4: Setting Token Expiry and Refreshing Tokens

Here's your content formatted in Markdown:

---

# Setting Token Expiry and Refreshing Tokens

Welcome back! In the previous lessons, you've learned how to set up a basic `/login` endpoint, generate JSON Web Tokens (JWT) upon successful login, and secure an endpoint using JWT in your Flask application. These are essential steps in building a secure API.

In this lesson, we'll go a step further by discussing the importance of setting token expiration times and introducing the concepts of **access tokens** and **refresh tokens**. Using these tokens effectively is crucial for maintaining the security and usability of your application.

---

## Recap of Existing Setup

Before diving into the new content, let's quickly recap our existing Flask and JWT setup to ensure we're all on the same page. Here's the fundamental configuration for our Flask app, including the mock database, JWT setup, and schema validation:

```python
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager
from marshmallow import Schema, fields, ValidationError
from marshmallow.validate import Length

# Initialize a Flask app instance
app = Flask(__name__)

# Mock database of users
database = [
    {"id": 1, "username": "cosmo", "password": "space-corgi"}
]

# Define a schema for validating login data
class LoginSchema(Schema):
    username = fields.Str(required=True, validate=Length(min=1))
    password = fields.Str(required=True, validate=Length(min=1))

# Set the secret key for signing JWTs
app.config['JWT_SECRET_KEY'] = 'super-secret'

# Initialize the JWTManager with the Flask app
jwt = JWTManager(app)
```

This setup initializes a Flask application, configures the JWT secret key, initializes the `JWTManager`, and sets up a mock database and a login schema for validation.

---

## Access Tokens vs Refresh Tokens

In previous lessons, we used **access tokens** to secure our endpoints when users logged in. Now, we'll introduce another type of token: the **refresh token**.

- **Access Tokens** are short-lived tokens used to authorize access to protected resources. They are included in the headers of API requests and are typically used on endpoints that require user authentication, such as `/profile`, `/dashboard`, or any other user-specific routes.
  
- **Refresh Tokens** are longer-lived and used to get new access tokens without requiring the user to log in again. They are used specifically on the token refresh route (e.g., `/refresh`) to obtain a new access token when the old one expires. Refresh tokens are not used directly to access resources but rather to acquire new access tokens.

By using refresh tokens, we can make our application more secure and user-friendly. Instead of forcing users to log in frequently, we can allow them to stay logged in by obtaining new access tokens automatically.

---

## Setting Duration Times

Setting token expiration times is critical for security. Tokens that never expire can be a significant security risk. By configuring these times, you reduce the window for potential misuse.

The `timedelta` class from Python's `datetime` module allows us to specify these durations. It accepts parameters like `seconds`, `minutes`, `hours`, `days`, and `weeks`.

Here are some examples of how to create different durations:

```python
from datetime import timedelta

# Examples of durations
seconds = timedelta(seconds=30)   # 30 seconds
minutes = timedelta(minutes=5)    # 5 minutes
hours = timedelta(hours=1)        # 1 hour
days = timedelta(days=1)          # 1 day
weeks = timedelta(weeks=1)        # 1 week
custom = timedelta(days=2, hours=3, minutes=15)  # 2 days, 3 hours, and 15 minutes
```

---

## Configuring Token Expiration Times

Now that we understand how to specify durations with `timedelta`, let's apply this to our token expiration configuration and ensure that tokens have a limited lifespan, enhancing security.

Here's how to configure the expiration times for access and refresh tokens:

```python
from datetime import timedelta

# Set the expiry time for access tokens (15 minutes)
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(minutes=15)
# Set the expiry time for refresh tokens (1 hour)
app.config['JWT_REFRESH_TOKEN_EXPIRES'] = timedelta(hours=1)
```

In this configuration:

- **Access Tokens** expire after 15 minutes.
- **Refresh Tokens** expire after 1 hour.

By setting these expiration times, you enforce periodic re-authentication and token renewal, which helps maintain the security of your application.

---

## Generating Access Tokens and Refresh Tokens

With our expiration times now set, let's extend our existing `/login` route to generate both tokens upon a successful login:

```python
from flask_jwt_extended import create_access_token, create_refresh_token

# Existing login route to include token generation
@app.route('/login', methods=['POST'])
def login():
    # -- previous input validation and user verification code goes here --
    
    # Check if the user exists and if the password matches
    if user and user["password"] == password:
        # Create a JWT access token
        access_token = create_access_token(identity=username)
        # Create a JWT refresh token
        refresh_token = create_refresh_token(identity=username)

        # Return the tokens as a JSON response
        return jsonify(access_token=access_token, refresh_token=refresh_token), 200
    else:
        # Return an error if the user does not exist or the password is incorrect
        return jsonify(error="Bad username or password"), 401
```

In this route, upon successful login:

- We generate an access token using `create_access_token(identity=username)`.
- We generate a refresh token using `create_refresh_token(identity=username)`.
- Both tokens are then returned in the response.

This setup ensures that the user receives both tokens needed for future access and token refresh actions.

---

## Creating a Token Refresh Route

Once the access token expires, users will need a way to get a new one without having to re-authenticate. This is where the **refresh token** comes into play.

Below is the implementation of a `/refresh` route that uses the refresh token to provide a new access token:

```python
from flask_jwt_extended import jwt_required, get_jwt_identity

# Define a refresh route that requires a valid refresh token to access
@app.route('/refresh', methods=['POST'])
@jwt_required(refresh=True)
def refresh():
    # Get the identity of the current user from the JWT refresh token
    current_user = get_jwt_identity()
    # Create a new access token
    new_access_token = create_access_token(identity=current_user)
    # Return the new access token as a JSON response
    return jsonify(access_token=new_access_token), 200
```

In this route:

- The `@jwt_required(refresh=True)` decorator ensures that the request includes a valid refresh token.
- `get_jwt_identity()` retrieves the identity (user) from the current refresh token.
- A new access token is generated using `create_access_token(identity=current_user)`.
- The new access token is returned in the response.

This process allows users to stay authenticated without having to provide their credentials every time their access token expires.

---

## Handling Expired Tokens

When an access token expires, the server responds with a `401 Unauthorized` status and a message like:

```json
{
    "msg": "Token has expired"
}
```

This indicates that the token is invalid due to expiration. The client should use the refresh token to get a new access token or prompt the user to re-authenticate, ensuring ongoing security.

---

## Summary and Next Steps

In this lesson, we covered:

- The importance of setting token expiration times for both access and refresh tokens.
- How to configure token expiration times in Flask.
- The differences between access tokens and refresh tokens.
- Generating both types of tokens upon login.
- Creating a route to refresh access tokens using a valid refresh token.

Congratulations on reaching the final stage of our course! Your dedication has brought you to a point where you can confidently secure Flask applications using JWT authentication. Up next, you have a few final tasks to complete that will reinforce these concepts.

You're almost there, so keep going strong!

---

## Understanding Token Expiry and Refreshing

Good job getting through the earlier lessons! Now, let's take a closer look at how your Flask application manages token expiry and refresh tokens.

All you need to do is run the code and observe its behavior. The tests include:

Logging in to receive both an access token and a refresh token.
Using the refresh token to get a new access token.
Accessing a protected route using the access token.
Be patient while the environment sets up the server and runs the necessary requests! You're doing well — keep it up! 🚀

```py
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, create_refresh_token, jwt_required, get_jwt_identity
from marshmallow import Schema, fields, ValidationError
from marshmallow.validate import Length
from datetime import timedelta

# Initialize a Flask app instance
app = Flask(__name__)

# Mock database of users
database = [
    {"id": 1, "username": "cosmo", "password": "space-corgi"}
]

# Define a schema for validating login data
class LoginSchema(Schema):
    username = fields.Str(required=True, validate=Length(min=1))
    password = fields.Str(required=True, validate=Length(min=1))

# Create an instance of LoginSchema
login_schema = LoginSchema()

# Set the secret key for signing JWTs
app.config['JWT_SECRET_KEY'] = 'super-secret' 

# Initialize the JWTManager with the Flask app
jwt = JWTManager(app)

# Set the expiry time for access tokens (15 minutes)
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(minutes=15)
# Set the expiry time for refresh tokens (1 hour)
app.config['JWT_REFRESH_TOKEN_EXPIRES'] = timedelta(hours=1)

# Define a route to receive login credentials
@app.route('/login', methods=['POST'])
def login():
    try:
        # Validate and deserialize the input data according to the schema
        data = login_schema.load(request.get_json())
    except ValidationError as err:
        # If validation fails, return an error message and a 400 status code
        return jsonify(error=err.messages), 400

    # Extract username and password from the validated data
    username = data['username']
    password = data['password']

    # Find the user in the mock database
    user = next((user for user in database if user["username"] == username), None)
    
    # Check if the user exists and if the password matches
    if user and user["password"] == password:
        # Create a JWT access token and refresh token
        access_token = create_access_token(identity=username)
        refresh_token = create_refresh_token(identity=username)
        # Return the tokens as a JSON response
        return jsonify(access_token=access_token, refresh_token=refresh_token), 200
    else:
        # Return an error if the user does not exist or the password is incorrect
        return jsonify(error="Bad username or password"), 401

# Define a refresh route that requires a valid refresh token to access
@app.route('/refresh', methods=['POST'])
@jwt_required(refresh=True)
def refresh():
    # Get the identity of the current user from the JWT refresh token
    current_user = get_jwt_identity()
    # Create a new access token
    new_access_token = create_access_token(identity=current_user)
    # Return the new access token as a JSON response
    return jsonify(access_token=new_access_token), 200

# Define a protected route that requires a valid JWT access token to access
@app.route('/protected', methods=['GET'])
@jwt_required()
def protected_route():
    return jsonify(message="This is a protected route, and you are authenticated!"), 200


```

## Generate and Refresh Tokens

Well done so far! Now, let's test your skills with a practical task.

Complete the existing /login and /refresh routes to ensure they work correctly with JWT in Flask. Fill in the missing parts of the code to:

Generate access and refresh tokens
Refresh the access token using the refresh token
This will reinforce your understanding and give you practical experience.

```py
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, create_refresh_token, jwt_required, get_jwt_identity
from marshmallow import Schema, fields, ValidationError
from marshmallow.validate import Length
from datetime import timedelta

# Initialize a Flask app instance
app = Flask(__name__)

# Mock database of users
database = [
    {"id": 1, "username": "cosmo", "password": "space-corgi"}
]

# Define a schema for validating login data
class LoginSchema(Schema):
    username = fields.Str(required=True, validate=Length(min=1))
    password = fields.Str(required=True, validate=Length(min=1))

# Create an instance of LoginSchema
login_schema = LoginSchema()

# Set the secret key for signing JWTs
app.config['JWT_SECRET_KEY'] = 'super-secret'

# Initialize the JWTManager with the Flask app
jwt = JWTManager(app)

# Set the expiry time for access tokens (15 minutes)
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(minutes=15)
# Set the expiry time for refresh tokens (1 hour)
app.config['JWT_REFRESH_TOKEN_EXPIRES'] = timedelta(hours=1)

# Define a route to receive login credentials
@app.route('/login', methods=['POST'])
def login():
    try:
        # Validate and deserialize the input data according to the schema
        data = login_schema.load(request.get_json())
    except ValidationError as err:
        # If validation fails, return an error message and a 400 status code
        return jsonify(error=err.messages), 400

    # Extract username and password from the validated data
    username = data['username']
    password = data['password']

    # Find the user in the mock database
    user = next((user for user in database if user["username"] == username), None)
    
    # Check if the user exists and if the password matches
    if user and user["password"] == password:
        # TODO: Create a JWT access token
        access_token = __________________(identity=username)
        # TODO: Create a JWT refresh token
        refresh_token = __________________(identity=username)
        # Return the tokens as a JSON response
        return jsonify(access_token=access_token, refresh_token=refresh_token), 200
    else:
        # Return an error if the user does not exist or the password is incorrect
        return jsonify(error="Bad username or password"), 401

# Define a refresh route that requires a valid refresh token to access
@app.route('/refresh', methods=['POST'])
@jwt_required(refresh=True)
def refresh():
    # TODO: Get the identity of the current user from the JWT refresh token
    current_user = __________________()
    # TODO: Create a new access token
    new_access_token = __________________(identity=current_user)
    # Return the new access token as a JSON response
    return jsonify(access_token=new_access_token), 200

# Define a protected route that requires a valid JWT access token to access
@app.route('/protected', methods=['GET'])
@jwt_required()
def protected_route():
    return jsonify(message="This is a protected route, and you are authenticated!"), 200

```

Here’s the complete implementation of the `/login` and `/refresh` routes using JWT in Flask. The missing parts of the code have been filled in, allowing it to generate access and refresh tokens as well as refresh the access token using the refresh token.

```py
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, create_refresh_token, jwt_required, get_jwt_identity
from marshmallow import Schema, fields, ValidationError
from marshmallow.validate import Length
from datetime import timedelta

# Initialize a Flask app instance
app = Flask(__name__)

# Mock database of users
database = [
    {"id": 1, "username": "cosmo", "password": "space-corgi"}
]

# Define a schema for validating login data
class LoginSchema(Schema):
    username = fields.Str(required=True, validate=Length(min=1))
    password = fields.Str(required=True, validate=Length(min=1))

# Create an instance of LoginSchema
login_schema = LoginSchema()

# Set the secret key for signing JWTs
app.config['JWT_SECRET_KEY'] = 'super-secret'

# Initialize the JWTManager with the Flask app
jwt = JWTManager(app)

# Set the expiry time for access tokens (15 minutes)
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(minutes=15)
# Set the expiry time for refresh tokens (1 hour)
app.config['JWT_REFRESH_TOKEN_EXPIRES'] = timedelta(hours=1)

# Define a route to receive login credentials
@app.route('/login', methods=['POST'])
def login():
    try:
        # Validate and deserialize the input data according to the schema
        data = login_schema.load(request.get_json())
    except ValidationError as err:
        # If validation fails, return an error message and a 400 status code
        return jsonify(error=err.messages), 400

    # Extract username and password from the validated data
    username = data['username']
    password = data['password']

    # Find the user in the mock database
    user = next((user for user in database if user["username"] == username), None)
    
    # Check if the user exists and if the password matches
    if user and user["password"] == password:
        # Create a JWT access token
        access_token = create_access_token(identity=username)
        # Create a JWT refresh token
        refresh_token = create_refresh_token(identity=username)
        # Return the tokens as a JSON response
        return jsonify(access_token=access_token, refresh_token=refresh_token), 200
    else:
        # Return an error if the user does not exist or the password is incorrect
        return jsonify(error="Bad username or password"), 401

# Define a refresh route that requires a valid refresh token to access
@app.route('/refresh', methods=['POST'])
@jwt_required(refresh=True)
def refresh():
    # Get the identity of the current user from the JWT refresh token
    current_user = get_jwt_identity()
    # Create a new access token
    new_access_token = create_access_token(identity=current_user)
    # Return the new access token as a JSON response
    return jsonify(access_token=new_access_token), 200

# Define a protected route that requires a valid JWT access token to access
@app.route('/protected', methods=['GET'])
@jwt_required()
def protected_route():
    return jsonify(message="This is a protected route, and you are authenticated!"), 200

if __name__ == "__main__":
    app.run(debug=True)
```

### Explanation of Changes:
1. **JWT Token Creation in `/login`:**
   - `create_access_token(identity=username)` generates the access token.
   - `create_refresh_token(identity=username)` generates the refresh token.

2. **Refreshing the Access Token in `/refresh`:**
   - `get_jwt_identity()` retrieves the identity (username) of the user from the refresh token.
   - `create_access_token(identity=current_user)` generates a new access token for the current user.

### Usage:
- **Login:** You send a POST request to `/login` with the username and password, and you receive an access token and a refresh token.
- **Refresh Token:** When the access token expires, you send a POST request to `/refresh` with the refresh token, and you get a new access token.
- **Protected Route:** Accessing `/protected` requires a valid access token.

## Fix the Token Required Bugs

You're doing great so far!

The provided Flask application has some issues with route decorators that need adjustment. Examine the @jwt_required decorators and ensure they're configured correctly, so the routes function as intended.

Keep pushing forward, and let's fix that code!

```py
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, create_refresh_token, jwt_required, get_jwt_identity
from marshmallow import Schema, fields, ValidationError
from marshmallow.validate import Length
from datetime import timedelta

# Initialize a Flask app instance
app = Flask(__name__)

# Mock database of users
database = [
    {"id": 1, "username": "cosmo", "password": "space-corgi"}
]

# Define a schema for validating login data
class LoginSchema(Schema):
    username = fields.Str(required=True, validate=Length(min=1))
    password = fields.Str(required=True, validate=Length(min=1))

# Create an instance of LoginSchema
login_schema = LoginSchema()

# Set the secret key for signing JWTs
app.config['JWT_SECRET_KEY'] = 'super-secret' 

# Initialize the JWTManager with the Flask app
jwt = JWTManager(app)

# Set the expiry time for access tokens (15 minutes)
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(minutes=15)
# Set the expiry time for refresh tokens (1 hour)
app.config['JWT_REFRESH_TOKEN_EXPIRES'] = timedelta(hours=1)

# Define a route to receive login credentials
@app.route('/login', methods=['POST'])
def login():
    try:
        # Validate and deserialize the input data according to the schema
        data = login_schema.load(request.get_json())
    except ValidationError as err:
        # If validation fails, return an error message and a 400 status code
        return jsonify(error=err.messages), 400

    # Extract username and password from the validated data
    username = data['username']
    password = data['password']

    # Find the user in the mock database
    user = next((user for user in database if user["username"] == username), None)
    
    # Check if the user exists and if the password matches
    if user and user["password"] == password:
        # Create a JWT access token and refresh token
        access_token = create_access_token(identity=username)
        refresh_token = create_refresh_token(identity=username)
        # Return the tokens as a JSON response
        return jsonify(access_token=access_token, refresh_token=refresh_token), 200
    else:
        # Return an error if the user does not exist or the password is incorrect
        return jsonify(error="Bad username or password"), 401

# Define a refresh route that requires a valid refresh token to access
@app.route('/refresh', methods=['POST'])
@jwt_required()
def refresh():
    # Get the identity of the current user from the JWT refresh token
    current_user = get_jwt_identity()
    # Create a new access token
    new_access_token = create_access_token(identity=current_user)
    # Return the new access token as a JSON response
    return jsonify(access_token=new_access_token), 200

# Define a protected route that requires a valid JWT access token to access
@app.route('/protected', methods=['GET'])
@jwt_required(refresh=True)
def protected_route():
    return jsonify(message="This is a protected route, and you are authenticated!"), 200
```
```py
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, create_refresh_token, jwt_required, get_jwt_identity
from marshmallow import Schema, fields, ValidationError
from marshmallow.validate import Length
from datetime import timedelta

# Initialize a Flask app instance
app = Flask(__name__)

# Mock database of users
database = [
    {"id": 1, "username": "cosmo", "password": "space-corgi"}
]

# Define a schema for validating login data
class LoginSchema(Schema):
    username = fields.Str(required=True, validate=Length(min=1))
    password = fields.Str(required=True, validate=Length(min=1))

# Create an instance of LoginSchema
login_schema = LoginSchema()

# Set the secret key for signing JWTs
app.config['JWT_SECRET_KEY'] = 'super-secret'

# Initialize the JWTManager with the Flask app
jwt = JWTManager(app)

# Set the expiry time for access tokens (15 minutes)
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(minutes=15)
# Set the expiry time for refresh tokens (1 hour)
app.config['JWT_REFRESH_TOKEN_EXPIRES'] = timedelta(hours=1)

# Define a route to receive login credentials
@app.route('/login', methods=['POST'])
def login():
    try:
        # Validate and deserialize the input data according to the schema
        data = login_schema.load(request.get_json())
    except ValidationError as err:
        # If validation fails, return an error message and a 400 status code
        return jsonify(error=err.messages), 400

    # Extract username and password from the validated data
    username = data['username']
    password = data['password']

    # Find the user in the mock database
    user = next((user for user in database if user["username"] == username), None)
    
    # Check if the user exists and if the password matches
    if user and user["password"] == password:
        # Create a JWT access token and refresh token
        access_token = create_access_token(identity=username)
        refresh_token = create_refresh_token(identity=username)
        # Return the tokens as a JSON response
        return jsonify(access_token=access_token, refresh_token=refresh_token), 200
    else:
        # Return an error if the user does not exist or the password is incorrect
        return jsonify(error="Bad username or password"), 401

# Define a refresh route that requires a valid refresh token to access
@app.route('/refresh', methods=['POST'])
@jwt_required(refresh=True)  # Ensure this route accepts refresh tokens only
def refresh():
    # Get the identity of the current user from the JWT refresh token
    current_user = get_jwt_identity()
    # Create a new access token
    new_access_token = create_access_token(identity=current_user)
    # Return the new access token as a JSON response
    return jsonify(access_token=new_access_token), 200

# Define a protected route that requires a valid JWT access token to access
@app.route('/protected', methods=['GET'])
@jwt_required()  # Only requires an access token for protected routes
def protected_route():
    return jsonify(message="This is a protected route, and you are authenticated!"), 200

if __name__ == "__main__":
    app.run(debug=True)


## Defining a Refresh Route

Nice work getting through the previous tasks!

This time, you'll define a refresh endpoint to verify the refresh token and generate a new access token.

Your task includes:

Defining a POST route at /refresh that requires a valid refresh token
Getting the identity of the current user from the refresh token
Creating a new access token
Returning the new access token as a JSON response with the key access_token
Keep pushing forward; you're almost there!

```py
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, create_refresh_token, jwt_required, get_jwt_identity
from marshmallow import Schema, fields, ValidationError
from marshmallow.validate import Length
from datetime import timedelta

# Initialize a Flask app instance
app = Flask(__name__)

# Mock database of users
database = [
    {"id": 1, "username": "cosmo", "password": "space-corgi"}
]

# Define a schema for validating login data
class LoginSchema(Schema):
    username = fields.Str(required=True, validate=Length(min=1))
    password = fields.Str(required=True, validate=Length(min=1))

# Create an instance of LoginSchema
login_schema = LoginSchema()

# Set the secret key for signing JWTs
app.config['JWT_SECRET_KEY'] = 'super-secret'

# Initialize the JWTManager with the Flask app
jwt = JWTManager(app)

# Set the expiry time for access tokens (15 minutes)
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(minutes=15)
# Set the expiry time for refresh tokens (1 hour)
app.config['JWT_REFRESH_TOKEN_EXPIRES'] = timedelta(hours=1)

# Define a route to receive login credentials
@app.route('/login', methods=['POST'])
def login():
    try:
        # Validate and deserialize the input data according to the schema
        data = login_schema.load(request.get_json())
    except ValidationError as err:
        # If validation fails, return an error message and a 400 status code
        return jsonify(error=err.messages), 400

    # Extract username and password from the validated data
    username = data['username']
    password = data['password']

    # Find the user in the mock database
    user = next((user for user in database if user["username"] == username), None)

    # Check if the user exists and if the password matches
    if user and user["password"] == password:
        # Create a JWT access token and refresh token
        access_token = create_access_token(identity=username)
        refresh_token = create_refresh_token(identity=username)
        # Return the tokens as a JSON response
        return jsonify(access_token=access_token, refresh_token=refresh_token), 200
    else:
        # Return an error if the user does not exist or the password is incorrect
        return jsonify(error="Bad username or password"), 401

# TODO: Define a POST route at '/refresh' that requires a valid refresh token to access

    # TODO: Get the identity of the current user from the JWT refresh token

    # TODO: Create a new access token using the identity of the current user

    # TODO: Return the new access token as a JSON response with the key "access_token" and a 200 status code



# Define a protected route that requires a valid JWT access token to access
@app.route('/protected', methods=['GET'])
@jwt_required()
def protected_route():
    return jsonify(message="This is a protected route, and you are authenticated!"), 200
```

The error you're encountering is caused by trying to import `jwt_refresh_token_required` from `flask_jwt_extended`, which no longer exists in recent versions of the library. The `jwt_refresh_token_required` decorator was removed and replaced with a different approach to handle refresh tokens.

### Solution:
Instead of using `jwt_refresh_token_required`, you should use `@jwt_required(refresh=True)` to specify that the route requires a refresh token.

Here is the corrected version of your `/refresh` endpoint:

```python
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity, create_refresh_token
from marshmallow import Schema, fields, ValidationError
from marshmallow.validate import Length
from datetime import timedelta

# Initialize a Flask app instance
app = Flask(__name__)

# Mock database of users
database = [
    {"id": 1, "username": "cosmo", "password": "space-corgi"}
]

# Define a schema for validating login data
class LoginSchema(Schema):
    username = fields.Str(required=True, validate=Length(min=1))
    password = fields.Str(required=True, validate=Length(min=1))

# Create an instance of LoginSchema
login_schema = LoginSchema()

# Set the secret key for signing JWTs
app.config['JWT_SECRET_KEY'] = 'super-secret'

# Initialize the JWTManager with the Flask app
jwt = JWTManager(app)

# Set the expiry time for access tokens (15 minutes)
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(minutes=15)
# Set the expiry time for refresh tokens (1 hour)
app.config['JWT_REFRESH_TOKEN_EXPIRES'] = timedelta(hours=1)

# Define a route to receive login credentials
@app.route('/login', methods=['POST'])
def login():
    try:
        # Validate and deserialize the input data according to the schema
        data = login_schema.load(request.get_json())
    except ValidationError as err:
        # If validation fails, return an error message and a 400 status code
        return jsonify(error=err.messages), 400

    # Extract username and password from the validated data
    username = data['username']
    password = data['password']

    # Find the user in the mock database
    user = next((user for user in database if user["username"] == username), None)

    # Check if the user exists and if the password matches
    if user and user["password"] == password:
        # Create a JWT access token and refresh token
        access_token = create_access_token(identity=username)
        refresh_token = create_refresh_token(identity=username)
        # Return the tokens as a JSON response
        return jsonify(access_token=access_token, refresh_token=refresh_token), 200
    else:
        # Return an error if the user does not exist or the password is incorrect
        return jsonify(error="Bad username or password"), 401

# Define a POST route at '/refresh' that requires a valid refresh token to access
@app.route('/refresh', methods=['POST'])
@jwt_required(refresh=True)  # Ensure that only refresh tokens can access this route
def refresh():
    # Get the identity of the current user from the refresh token
    current_user = get_jwt_identity()

    # Create a new access token using the identity of the current user
    new_access_token = create_access_token(identity=current_user)

    # Return the new access token as a JSON response with the key "access_token" and a 200 status code
    return jsonify(access_token=new_access_token), 200

# Define a protected route that requires a valid JWT access token to access
@app.route('/protected', methods=['GET'])
@jwt_required()
def protected_route():
    return jsonify(message="This is a protected route, and you are authenticated!"), 200

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)
```

### Key Changes:
- **`@jwt_required(refresh=True)`**: This is the new way to require a refresh token to access the `/refresh` endpoint. It ensures that only refresh tokens are allowed.
  
Now, this should resolve the issue, and your Flask app should correctly handle refresh token validation and creation of new access tokens.

Now it's time to put everything you've learned into practice.

Your task is to write a Flask application that handles user login, token generation, and token refreshing.

Set expiration times:
Access tokens: 20 minutes
Refresh tokens: 2 hours
Generate access and refresh tokens upon successful login.
Return a JSON response with the keys: access_token and refresh_token
Create a /refresh route to obtain a new access token using the refresh token.
Return the new access token as a JSON response with the key: access_token
You're almost at the finish line!

```py
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, create_refresh_token, jwt_required, get_jwt_identity
from marshmallow import Schema, fields, ValidationError
from marshmallow.validate import Length
from datetime import timedelta

# Initialize a Flask app instance
app = Flask(__name__)

# Mock database of users
database = [
    {"id": 1, "username": "luna", "password": "moon-paw"}
]

# Define a schema for validating login data
class LoginSchema(Schema):
    username = fields.Str(required=True, validate=Length(min=1))
    password = fields.Str(required=True, validate=Length(min=1))

# Create an instance of LoginSchema
login_schema = LoginSchema()

# Set the secret key for signing JWTs
app.config['JWT_SECRET_KEY'] = 'super-secret'

# Initialize the JWTManager with the Flask app
jwt = JWTManager(app)

# TODO: Set the expiry time for access tokens (20 minutes)

# TODO: Set the expiry time for refresh tokens (2 hours)

# Define a route to receive login credentials
@app.route('/login', methods=['POST'])
def login():
    try:
        # Validate and deserialize the input data according to the schema
        data = login_schema.load(request.get_json())
    except ValidationError as err:
        # If validation fails, return an error message and a 400 status code
        return jsonify(error=err.messages), 400

    # Extract username and password from the validated data
    username = data['username']
    password = data['password']

    # Find the user in the mock database
    user = next((user for user in database if user["username"] == username), None)
    
    # Check if the user exists and if the password matches
    if user and user["password"] == password:
        # TODO: Create a JWT access token
 
        # TODO: Create a JWT refresh token

        # TODO: Return a JSON response with the keys: access_token and refresh_token, and a 200 status code

    else:
        # Return an error if the user does not exist or the password is incorrect
        return jsonify(error="Bad username or password"), 401

# TODO: Define a POST route at '/refresh' that requires a valid refresh token to access

    # TODO: Get the identity of the current user from the JWT refresh token

    # TODO: Create a new access token using the identity of the current user

    # TODO: Return the new access token as a JSON response with the key "access_token" and a 200 status code


# Define a protected route that requires a valid JWT access token to access
@app.route('/protected', methods=['GET'])
@jwt_required()
def protected_route():
    return jsonify(message="This is a protected route, and you are authenticated!"), 200
```

Here’s the complete Flask application to handle user login, token generation, and token refreshing:

```python
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, create_refresh_token, jwt_required, get_jwt_identity
from marshmallow import Schema, fields, ValidationError
from marshmallow.validate import Length
from datetime import timedelta

# Initialize a Flask app instance
app = Flask(__name__)

# Mock database of users
database = [
    {"id": 1, "username": "luna", "password": "moon-paw"}
]

# Define a schema for validating login data
class LoginSchema(Schema):
    username = fields.Str(required=True, validate=Length(min=1))
    password = fields.Str(required=True, validate=Length(min=1))

# Create an instance of LoginSchema
login_schema = LoginSchema()

# Set the secret key for signing JWTs
app.config['JWT_SECRET_KEY'] = 'super-secret'

# Initialize the JWTManager with the Flask app
jwt = JWTManager(app)

# Set the expiry time for access tokens (20 minutes)
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(minutes=20)

# Set the expiry time for refresh tokens (2 hours)
app.config['JWT_REFRESH_TOKEN_EXPIRES'] = timedelta(hours=2)

# Define a route to receive login credentials
@app.route('/login', methods=['POST'])
def login():
    try:
        # Validate and deserialize the input data according to the schema
        data = login_schema.load(request.get_json())
    except ValidationError as err:
        # If validation fails, return an error message and a 400 status code
        return jsonify(error=err.messages), 400

    # Extract username and password from the validated data
    username = data['username']
    password = data['password']

    # Find the user in the mock database
    user = next((user for user in database if user["username"] == username), None)
    
    # Check if the user exists and if the password matches
    if user and user["password"] == password:
        # Create a JWT access token
        access_token = create_access_token(identity=username)
        
        # Create a JWT refresh token
        refresh_token = create_refresh_token(identity=username)

        # Return a JSON response with the keys: access_token and refresh_token, and a 200 status code
        return jsonify(access_token=access_token, refresh_token=refresh_token), 200
    else:
        # Return an error if the user does not exist or the password is incorrect
        return jsonify(error="Bad username or password"), 401

# Define a POST route at '/refresh' that requires a valid refresh token to access
@app.route('/refresh', methods=['POST'])
@jwt_required(refresh=True)  # Ensure that only refresh tokens can access this route
def refresh():
    # Get the identity of the current user from the JWT refresh token
    current_user = get_jwt_identity()

    # Create a new access token using the identity of the current user
    new_access_token = create_access_token(identity=current_user)

    # Return the new access token as a JSON response with the key "access_token" and a 200 status code
    return jsonify(access_token=new_access_token), 200

# Define a protected route that requires a valid JWT access token to access
@app.route('/protected', methods=['GET'])
@jwt_required()
def protected_route():
    return jsonify(message="This is a protected route, and you are authenticated!"), 200

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)
```

### Key Features:
1. **Access Token Expiration**: Access tokens are set to expire in 20 minutes.
2. **Refresh Token Expiration**: Refresh tokens are set to expire in 2 hours.
3. **Login Route** (`/login`): Verifies the user, generates access and refresh tokens, and returns them.
4. **Refresh Route** (`/refresh`): Requires a valid refresh token to generate a new access token.
5. **Protected Route** (`/protected`): Can only be accessed with a valid access token.

You can now test the application by:
- Sending a POST request to `/login` with valid credentials to receive access and refresh tokens.
- Using the refresh token in a POST request to `/refresh` to get a new access token.